In [22]:
import concurrent.futures
import time

def fn_take_time(seconds=3):
    print('before execution')
    time.sleep(seconds)
    print('after execution')
    return str(seconds) + "s"

In [19]:
%%time
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as ex:
    future = ex.map(fn_take_time, [1,2,3,4,11,1,2,3,4,1])
    time.sleep(2)
    print("After sleep calling result")

before execution
before execution
before execution
before execution
before execution
before execution
before execution
before execution
before execution
before execution


after execution
after execution
after execution
after execution
after execution
After sleep calling result
after execution
after execution
after execution
after execution
after execution
CPU times: user 14.8 ms, sys: 1.08 ms, total: 15.9 ms
Wall time: 11 s


In [23]:
timer=[1,2,3,4,11,1,2,3,4,1]

In [24]:
%%time
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as ex:
    results = {
        ex.submit(fn_take_time, the_time): the_time for the_time in timer
    }

    for result in concurrent.futures.as_completed(results):
        exchange = results[result]
        print(result.result())

before execution
before execution
before execution
before execution
before execution
before execution
before execution
before execution
before execution
before execution


after execution
1s
after execution
1s
after execution
1s
after execution
2s
after execution
2s
after execution
3s
after execution
3s
after execution
4s
after execution
4s
after execution
11s
CPU times: user 12.6 ms, sys: 791 µs, total: 13.4 ms
Wall time: 11 s


In [ ]:
import requests

def make_multithreaded(func: Callable,
                       threads: int) -> Callable:
    """Creates a multithreaded version of a function.

    Args:
        func: Function that is meant to be executed on
            a list of input objects.
        threads: The number of threads the multithreaded
            version of the function should use.

    Returns:
        A multithreaded version of the `func` input function.

    """

    def multithreaded_function(input_list: list, *args) -> list:
        """Executes function on input list using multiple threads.

        Args:
            input_list: The list of objects a function should be
                executed on.
            *args: Variable length argument list of additional
                parameters needed for the function to be executed.

        Returns:
            List of results after all input list elements were
            processed. Input order might not be preserved in
            output list.

        """
        results = []

        with futures.ThreadPoolExecutor(max_workers=threads) as executor:
            future_to_input_item = {
                executor.submit(
                    func,
                    input_item,
                    *args
                ): input_item for input_item in input_list
            }

            for future in futures.as_completed(future_to_input_item):
                result = future.result()

                if isinstance(result, list):
                    results.extend(result)
                else:
                    results.append(result)

        return results

    return multithreaded_function


input_list = ["https://httpbin.org/delay/10"]*10
response = requests.get("https://httpbin.org/delay/10")


with futures.ThreadPoolExecutor(max_workers=threads) as executor:
    future_to_input_item = {
        executor.submit()
    }
print(response.json())

In [ ]:
# Credit: https://github.com/michaelharms/comcrawl/blob/a89236080c5e7f4ce6a2e0d39c5f59671f22181e/comcrawl/utils/multithreading.py#L12

# note: this script is very useful way to make the requests session to be multithreading 
# and easily called simply by scraper_func(session, url, params, headers, threads=4)
``
# needs more work to investigate it

"""Multithreading Helpers.

This module contains utility functions
to manage multi-threading.

"""

from typing import Callable, List
from concurrent import futures
import json
import requests


def search_single_index(url: str):
    response = requests.get(url)
    if response.status_code == 200:
        results = [
            json.loads(result) for result in response.content.splitlines()
        ]
    return results


def search_multiple_indexes(url: str,
                            indexes: list[str],
                            threads: int = None) -> list[dict]:
    results = []

    # multi-threaded search
    if threads:
        mulithreaded_search = make_multithreaded(search_single_index,
                                                 threads)
        results = mulithreaded_search(indexes, url)

    # single-threaded search
    else:
        for index in indexes:
            index_results = search_single_index(index, url)
            results.extend(index_results)

    return results


def make_multithreaded(func: Callable,
                       threads: int) -> Callable:
    """Creates a multithreaded version of a function.

    Args:
        func: Function that is meant to be executed on
            a list of input objects.
        threads: The number of threads the multithreaded
            version of the function should use.

    Returns:
        A multithreaded version of the `func` input function.

    """

    def multithreaded_function(input_list: List, *args) -> List:
        """Executes function on input list using multiple threads.

        Args:
            input_list: The list of objects a function should be
                executed on.
            *args: Variable length argument list of additional
                parameters needed for the function to be executed.

        Returns:
            List of results after all input list elements were
            processed. Input order might not be preserved in
            output list.

        """
        results = []

        with futures.ThreadPoolExecutor(max_workers=threads) as executor:
            future_to_input_item = {
                executor.submit(
                    func,
                    input_item,
                    *args
                ): input_item for input_item in input_list
            }

            for future in futures.as_completed(future_to_input_item):
                result = future.result()

                if isinstance(result, List):
                    results.extend(result)
                else:
                    results.append(result)

        return results

    return multithreaded_function

In [ ]:
search_multiple_indexes(url=)